In [2]:
import pandas as pd
import numpy 
import regex as re
import random
import sklearn


import scipy



In [3]:
pheno = pd.read_table("pheno.txt")
data = pd.read_table("normalized_data.txt")

pheno.head()


,ParticipantID,Phenotype,Sample_Source,PlateId,Sex,HybControlNormScale,RowCheck,NormScale_20,NormScale_0_005,NormScale_0_5,ANMLFractionUsed_20,ANMLFractionUsed_0_005,ANMLFractionUsed_0_5
0,C1,Control,Blood,P0023923,F,0.947480,PASS,0.769958,0.813128,0.818578,0.868,0.838,0.745
1,C2,Control,Blood,P0023895,F,0.891341,PASS,0.765640,0.967612,0.891220,0.874,0.900,0.790
2,C3,Control,Blood,P0023895,F,0.888196,PASS,0.412029,0.918688,0.810701,0.732,0.881,0.666
3,C4,Control,Blood,P0023923,F,0.856026,PASS,0.774548,0.866460,0.810048,0.879,0.875,0.777
4,C5,Control,Blood,P0023895,F,0.982222,PASS,0.665371,0.833492,0.820785,0.804,0.900,0.852


In [4]:
data.head()

metadata = data.iloc[:, :4]

metadata

,Molecule,Protein,Gene,UniProt
0,10000-28,CRBB2,CRYBB2,P43320
1,10001-7,c-Raf,RAF1,P04049
2,10003-15,ZNF41,ZNF41,P51814
3,10006-25,ELK1,ELK1,P19419
4,10008-43,GUC1A,GUCA1A,P43080
...,...,...,...,...
5279,9993-11,ZN264,ZNF264,O43296
5280,9994-217,ATP4B,ATP4B,P51164
5281,9995-6,DUT,DUT,P33316
5282,9997-12,UBXN4,UBXN4,Q92575


In [5]:
data = data.transpose()



In [6]:
data.columns = data.iloc[0]
data.drop(data.index[0:4], inplace=True)





In [7]:
data = data.apply(pd.to_numeric)

In [8]:
data.dtypes

Molecule
10000-28    float64
10001-7     float64
10003-15    float64
10006-25    float64
10008-43    float64
             ...   
9993-11     float64
9994-217    float64
9995-6      float64
9997-12     float64
9999-1      float64
Length: 5284, dtype: object

In [9]:
target = data.index.tolist()

#replace controls with 0 and patient with 1

target = [0 if re.search('[C]', x) else 1 for x in target ]


In [10]:
data["target"] = target
data['target']

C1     0
C2     0
C3     0
C4     0
C5     0
C8     0
C9     0
C11    0
C14    0
C15    0
C16    0
C17    0
C18    0
C19    0
C20    0
C21    0
C24    0
C25    0
C26    0
C27    0
P1     1
P2     1
P4     1
P6     1
P9     1
P11    1
P13    1
P17    1
P18    1
P19    1
P20    1
P21    1
P24    1
P25    1
P27    1
P28    1
P29    1
P30    1
P31    1
P32    1
Name: target, dtype: int64

## Feature Selection

In [11]:
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

In [12]:
X = data.drop(["target"],axis=1)
y = data['target']

#Using Pearson Correlation
#plt.figure()
cor = data.corr()


In [13]:
#sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
#plt.show()

#Correlation with output variable
cor_target = abs(cor["target"])#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.5]


In [14]:
rel_data = data[[x for x in relevant_features.index]]
rel_data

Molecule,10440-26,11696-7,12800-5,13463-1,14008-22,14054-17,14131-37,16915-153,18819-21,18871-24,...,2615-60,2654-19,2944-66,3073-51,4374-45,5134-52,6375-75,7776-20,8483-5,target
C1,1012.6,10152.1,4794.6,1210.1,1006.0,7789.0,5362.8,11469.4,3487.7,1296.5,...,2922.8,13382.9,1181.7,4035.6,16061.5,2704.2,2197.5,8136.8,96.9,0
C2,870.8,8245.5,4726.7,970.7,1070.2,6609.3,4890.4,8125.7,6824.6,1330.0,...,2735.6,9179.1,1081.5,4357.8,11091.8,2488.9,1980.3,9561.3,107.2,0
C3,953.5,6604.4,2119.7,1105.6,791.1,3910.2,3518.1,11253.5,7253.6,996.2,...,1723.7,6556.5,1047.1,3326.8,7473.5,1791.7,1651.5,4944.6,118.8,0
C4,1075.5,7772.7,4769.7,1240.0,1002.7,7868.4,5953.9,10975.6,6196.9,1521.1,...,2924.8,12731.9,1066.6,5208.8,15619.9,3511.5,2047.3,8012.1,99.6,0
C5,1181.2,10094.5,4297.4,1232.1,1039.6,7901.5,5596.9,8758.8,7051.5,1461.0,...,2818.8,11596.8,1174.2,5338.6,12462.0,3565.5,2029.0,9353.2,103.0,0
C8,998.7,8086.3,4106.6,1175.8,1027.9,6392.8,5112.2,8576.8,7742.0,1356.8,...,2782.7,10338.4,1123.6,4355.2,12649.9,2729.3,2198.3,8744.2,125.9,0
C9,1105.4,8161.1,3687.1,1763.6,909.6,5981.2,6162.2,13312.2,8168.4,1410.4,...,2953.0,11457.2,1212.3,4593.6,14561.6,2887.0,2058.3,8128.1,112.1,0
C11,1092.0,6620.6,3173.5,1029.7,1011.7,7584.5,4616.8,11967.0,7676.9,1339.6,...,2751.3,9690.3,1208.4,3977.8,9395.5,2846.5,2229.1,8320.1,92.9,0
C14,1050.7,10211.3,4660.2,1236.4,1044.3,6115.0,6344.0,8130.5,6712.1,1350.9,...,2949.2,10917.2,1283.6,4135.3,11969.9,2664.3,2263.2,9317.4,107.9,0
C15,1056.7,9012.6,4125.1,1122.5,1238.2,7585.5,7002.7,10619.4,8205.1,1386.5,...,3554.6,11095.9,1354.8,4228.1,15890.3,2717.6,2585.7,10966.9,97.1,0


### Fitting GLM

In [15]:
cases = rel_data[rel_data["target"]==1]
controls = rel_data[rel_data["target"]!=1]


def simulate_protein_data(df):
    means = df.mean(axis=0).tolist()
    sds = df.std(axis=0).tolist()

    if len(means) != len(sds): ## sanity check
        return "List Imbalance"
    
    simulated_data = []
    for _ in range(1000):
        col = []
        for i in range(len(means)):
            col.append(random.gauss(means[i],sds[i]))
        simulated_data.append(col)
        

    simulated_dataframe = pd.DataFrame(simulated_data)
    return simulated_dataframe 




Molecule,10440-26,11696-7,12800-5,13463-1,14008-22,14054-17,14131-37,16915-153,18819-21,18871-24,...,2615-60,2654-19,2944-66,3073-51,4374-45,5134-52,6375-75,7776-20,8483-5,target


In [16]:
cases_simul = simulate_protein_data(cases)

In [22]:
import numpy as np
print(np.median(cases_simul.iloc[:,0]))
np.median(cases.iloc[:,0])

1215.6386010651995


1163.65

Molecule,10440-26,11696-7,12800-5,13463-1,14008-22,14054-17,14131-37,16915-153,18819-21,18871-24,...,2615-60,2654-19,2944-66,3073-51,4374-45,5134-52,6375-75,7776-20,8483-5,target
P1,1325.5,14114.5,6896.4,1689.8,1142.8,11714.8,7337.8,8149.4,10893.0,1802.7,...,3969.8,16302.7,1595.9,4668.3,22719.9,3680.7,2784.2,12815.0,85.6,1
P2,976.2,10277.4,5967.6,1532.5,1126.0,5805.9,5198.2,7447.5,8884.6,1540.9,...,3248.7,12700.9,1369.1,3855.1,14232.8,3545.0,2580.0,8935.5,89.7,1
P4,1281.0,8710.4,4311.7,1251.2,1251.2,9377.5,7402.7,10930.7,5976.9,1329.5,...,3514.0,10038.5,1298.3,5561.2,8737.7,3352.5,2451.8,8944.6,106.3,1
P6,1450.5,24317.7,5401.7,2665.1,1339.9,8589.6,9233.7,8243.0,10696.6,2043.7,...,4302.3,18949.6,1892.4,6063.8,17745.0,4129.7,3209.9,11669.1,93.0,1
P9,1088.2,10313.4,3229.0,1341.1,2258.9,8932.1,5953.9,7133.1,9169.3,1763.6,...,2878.9,12791.3,1440.5,4118.5,15281.9,3543.8,2271.1,9924.4,101.3,1
P11,1767.8,12882.6,6078.9,3200.2,1056.9,14021.8,7533.4,7409.2,11174.0,2053.2,...,3677.0,16756.6,1886.3,5671.6,20123.2,4118.0,2596.6,10753.1,89.3,1
P13,1055.0,13676.3,4741.4,1449.6,1039.1,7023.7,6040.0,9346.9,10531.1,1530.2,...,3099.6,12687.9,1583.8,4520.7,15700.5,3083.2,2903.5,8534.6,114.3,1
P17,1011.4,12353.8,6269.3,1500.9,1078.8,9527.3,6683.7,7825.2,8473.5,2023.7,...,3173.3,12069.2,1216.2,5080.6,13369.3,4592.0,2001.6,10800.1,101.5,1
P18,1155.3,14602.5,5795.4,1520.7,1156.2,10512.2,7753.1,6814.1,8781.0,1853.8,...,3678.6,15226.8,1451.3,7522.3,20664.9,3928.3,2343.7,11407.7,83.0,1
P19,1092.7,11887.1,5986.4,1548.6,1307.2,7815.2,6420.2,7155.4,8598.0,1881.2,...,3726.1,13101.1,1538.8,5367.9,16547.8,3256.4,2879.2,11103.6,96.0,1
